In [2]:
%load_ext autoreload
%autoreload 2

import os, json
import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl

from ablang_train import ABtokenizer, AbLang, TrainingFrame, CallbackHandler, AbDataModule, ablang_parse_args

/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# SET ARGUMENTS AND HPARAMS
arguments = ablang_parse_args(args="", is_test=True)

arguments.model_specific_args.n_encoder_blocks = 1
arguments.model_specific_args.hidden_embed_size = 64
arguments.model_specific_args.n_attn_heads = 8
arguments.model_specific_args.over_sample_data = 0
arguments.model_specific_args.cpus = 1
arguments.model_specific_args.data_path = '../data/single_data/' #'/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-human/' #
arguments.model_specific_args.train_batch_size = 10
arguments.model_specific_args.accumulate_grad_batches = 1
arguments.model_specific_args.effective_batch_size = 10
arguments.model_specific_args.eval_batch_size = 10
arguments.model_specific_args.val_check_interval = 1
arguments.model_specific_args.use_tkn_dropout = False
arguments.model_specific_args.learning_rate = 4e-02 #4e-04
arguments.model_specific_args.change_percent = 1
arguments.model_specific_args.leave_percent = 0 #4e-04
arguments.model_specific_args.loss_fn = "Focal_Loss" #"CrossEntropy_Loss" #"Focal_Loss"
arguments.model_specific_args.a_fn = "gelu"
#arguments.model_specific_args.fl_gamma = 5
#arguments.model_specific_args.out_path = #set path

In [3]:
arguments = ablang_parse_args(args=["--json_args", "../reports/model_arguments/05_paired_C.json"], is_test=True)
arguments.model_specific_args.train_batch_size = 10
arguments.model_specific_args

Namespace(name='paired_C', json_args='../reports/model_arguments/05_paired_C.json', n_encoder_blocks=6, hidden_embed_size=320, n_attn_heads=20, dropout=0.1, use_tkn_dropout=False, loss_fn='Focal_Loss', a_fn='swiglu', fl_gamma=2, mask_percent=0.15, variable_masking=False, mask_technique='random', change_percent=0.8, leave_percent=0.1, initializer_range=0.02, layer_norm_eps=1e-12, data_path='/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/', out_path='/data/iraqbabbler/olsen/Documents/projects/AbLang/model-catalogue/paired-ablang/train_ablang_pair/reports/models', cpus=1, max_fit_batch_size=300, effective_batch_size=16, num_training_steps=10000, warmup_steps=2000, learning_rate=0.0004, cdr3_focus=1, weight_decay=0.01, adam_epsilon=1e-08, adam_betas=[0.9, 0.98], seed=42, eval_batch_size=100, over_sample_data=0, logger=True, enable_checkpointing=True, default_root_dir=None, gradient_clip_val=None, gradient_clip_algorithm=None, num_nodes=1, num_processes=None, devices=No

In [4]:
def set_neptune_logger(args):
    """
    Initialize Neptune logger
    """

    neptune_args = { 'api_key':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0N2Y2YmIxMS02OWM3LTRhY2MtYTQxOC0xODU5N2E0ODFmMzEifQ==",
    'project':"tobiasheol/AbLangTraining",
    'name':args.name,
    'log_model_checkpoints':False,
    }

    return pl.loggers.neptune.NeptuneLogger(**neptune_args)

arguments.trainer_args['logger'] = set_neptune_logger(arguments.model_specific_args)
arguments.trainer_args['log_every_n_steps'] = 1


callbacks = CallbackHandler(save_step_frequency=1, 
                                progress_refresh_rate=0, 
                                outpath=arguments.model_specific_args.out_path)

In [5]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    # For atomic operations there is currently
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    #
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    np.random.seed(seed)
    
    pl.seed_everything(seed)
    
# SET SEED - IMPORTANT FOR MULTIPLE GPUS, OTHERWISE GOOD FOR REPRODUCIBILITY
enforce_reproducibility(arguments.model_specific_args.seed)

Global seed set to 42


In [6]:
# LOAD AND INITIATE DATA
arguments.model_specific_args.mask_technique = 'random'

ablang_dm = AbDataModule(arguments.model_specific_args, ABtokenizer) 
# You are supposed to just be able to add abrep to the fit function, but it doesn't work when using multiple GPUs
ablang_dm.setup('fit')

train = ablang_dm.train_dataloader()
val = ablang_dm.val_dataloader()

In [7]:
for batch in train:
    
    print(batch['input'][0])
    print(batch['labels'][:len(batch['input'][0])])
    break

tensor([ 0,  6, 15, 10,  7, 15,  6,  7, 12, 12, 12, 16,  8,  4, 13, 12,  2,  7,
        20, 23, 20,  7, 18,  8, 14,  7, 17, 17,  8, 16,  1,  5, 18, 23, 23,  7,
        19, 17, 13, 10, 14, 13, 12,  7, 12, 16,  6, 19, 15, 12, 17, 16,  2,  7,
         4, 14, 18, 10,  6, 14,  8, 10, 18, 14, 14,  7, 15,  4, 12,  2, 17,  8,
        16,  7,  2,  5,  5, 17,  4,  7, 16, 14, 15, 20, 10,  1,  9,  7, 20,  2,
         6,  6,  5,  8, 14, 15, 18, 18, 11,  8,  2, 23, 12, 17,  4,  7,  4, 18,
        12, 13, 18, 18, 18, 16,  1,  5,  6, 10, 12, 10, 19,  3,  8, 15,  8, 15,
         7,  2, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 

In [8]:
# LOAD MODEL
model = TrainingFrame(arguments.model_specific_args, AbLang, ABtokenizer)

# INITIALISE TRAINER
trainer = pl.Trainer(**arguments.trainer_args, callbacks=callbacks())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.


In [ ]:
trainer.fit(model, train, val)


  | Name    | Type       | Params
---------------------------------------
0 | loss_fn | Focal_Loss | 0     
1 | ablang  | AbLang     | 10.1 M
---------------------------------------
10.1 M    Trainable params
48        Non-trainable params
10.1 M    Total params
40.313    Total estimated model params size (MB)


https://app.neptune.ai/tobiasheol/AbLangTraining/e/ABLANG-507


Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/help/nvml_error/


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/neptune/new/attributes/attribute.py:64: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` instead.
  return self.assign(value, wait)
/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does no

In [ ]:
seq_to_restore = '<EVQLVESGPGLVQPGKSLRLSCVASGFTFSGYGMHWVRQAPGKGLEWIALIIYDESNKYYADSVKGRFTISRDNSKNTLYLQMSSLRAEDTAVFYCAKVKFYDPTAPNDYWGQGTLVTVSS>|'

In [ ]:
tokenizer1.aa_to_token

In [ ]:
model1 = model.ablang
tokenizer1 = model.tokenizer

with torch.no_grad():
    tokenPreds = model1(tokenizer1([seq_to_restore], pad=True, w_extra_tkns=False, device='cpu'))

tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)

aaPreds = tokenizer1(tokenMAX.indices, mode='decode', device='cpu')

unkMatrix = torch.zeros(tokenMAX[0].shape, dtype=torch.long, device='cpu') + 21

aaPreds50 = ['-'.join(tokenizer1(torch.where(tokenMAX[0]<=.5, unkMatrix, tokenMAX[1]).detach(), mode='decode')[0].split('<unk>'))]

In [ ]:
tokenPreds[0][2]

In [ ]:
tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)
tokenMAX[1]

In [ ]:
masked_to_restore = '***LVESGPGLVQPGKSLRLSCVASGFTFSGYGMHWVRQAPGKGLEWIALIIYDESNKYYADSVKGRFTISRDNSKNTLYLQMSSLRAEDTAVFYCAKVKFYDPTAPND************'
masked_to_restore = '<********************************GMHWVRQAPGKGLEWIALI*********ADSVKGRFTISRDNSKNTLYLQMSSLRA*********************************>|'

In [ ]:
with torch.no_grad():
    tokenPreds = model1(tokenizer1([masked_to_restore], pad=True, w_extra_tkns=False, device='cpu'))
    tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)
    masked_aaPreds = tokenizer1(tokenMAX[1], mode='decode')
masked_aaPreds

In [ ]:
print(aaPreds[0])
print(aaPreds50[0])
print(seq_to_restore)